# CoNLL MAIN PROGRAM
## Encoder-Decoder with Attention
### Current archetype: BiGRU with Attention
--------------------------
#### Plan:
1. Finish commenting -- **DONE**
2. Edit evaluation functions -- **DONE**
3. Test with current design (get dev data and test on dev data) -- **DONE**
4. Incorporate tags in input -- **DONE**
5. Test with tags in input -- **DONE**
6. Convert GRU to LSTM -- **CANCELED**
7. Test with LSTM structure -- **CANCELED**
8. Edit for BiRNN structure (GRU) -- **DONE**
9. Test with fully developed structure
10. Discuss future possibilities
--------------------------
#### Questions:
1. What does .view() do? **Answer**: resizes tensors (see section 2 of tests)
2. How does super() work with pytorch? **Answer**: Basic inheritance
3. What is squeeze/unsqueeze? **Answer**: Squeeze squishes any one-sized dims in a tensor, unsqueeze adds a 1-sized dim at a given position in the tensor
4. What does .size() do? **Answer**: returns size of tensor
5. What is topk? **Answer**: Returns top k largest values of tensor along a specified dim
--------------------------
#### Need help with spots:
1. Line 38 in "# Training loop definition": How to resize tensors so they can be concatenated? Purpose: to concatenate context (morphological tags) tensor to decoder hidden state (this is where it is initiated) -- **FIXED**
2. In "# Decoder RNN with attention" and elsewhere: How to reshape vectors such that decoder hidden state with context included can be run through torch.bmm with encoder output without losing information? -- **FIXED**

In [ ]:
'''      ANNOTATION KEY      '''
# S/E = self explanatory
# ??? = uncertain of meaning/purpose
# TUOP = textual update of progress

In [1]:
from __future__ import unicode_literals, print_function, division # ???
from io import open # For opening files
import unicodedata # To convert to ASCII -- necessary?
import string # ???
import re # For normalizing text -- necessary?
import random # For randomizing samples of data

import torch # Imports pytorch
import torch.nn as nn # Imports nn from pytorch
from torch import optim # For optimization (SGD)
import torch.nn.functional as F # For linear functions in neural nets

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Sets what processor to use (GPU/CPU)

In [2]:
# Sets up language classes for indexing characters

# Start and end of word tokens
SOW_token = 0
EOW_token = 1

# Class to set up lemmas and inflected words -- find new name maybe?
class Lang:
    def __init__(self, name):
        self.name = name # S/E
        # Creates an index for char --> index, char count, and index --> char
        #   plus total number of unique chars
        self.char2index = {}
        self.char2count = {}
        self.index2char = {0: "SOW", 1: "EOW"}
        self.n_chars = 2  # Count SOW and EOW

    # Adds all characters in word to Lang
    def addWord(self, word, tags=None):
        word = list(word)
        if tags != None:
            word += tags
        for char in word:
            self.addChar(char)

    # Adds char information to/creates indexes and counts for Lang
    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

In [56]:
# Reads in language and setting
# Outputs lemma Lang, inflected word Lang, and pairs plus tags

def readLangs(lang, setting):
    print("Reading lines...")

    # Read the file based on input language and setting (low/med/high) and split into lines
    lines = open('./all/%s-%s' % (lang, setting), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs of lemmas/words and tags; normalize
    pairs = [[s for s in l.split('\t')[:2]] for l in lines] # Makes lemma/word pairs
    tags = [l.split('\t')[2].split(';') for l in lines] # Makes list of tags to add to input
    pairs_tags = list(zip(pairs,tags)) # Zips matching lemma/word pairs and tag set

    # Make Lang instances
    lemmas = Lang("lemmas") # S/E
    inflected_words = Lang("inflected words") # S/E

    return lemmas, inflected_words, pairs_tags # S/E

In [57]:
# Sets up lemmas, words, lemma/word-pairs and tags pairs, and maximum string length

def prepareData(lang, setting):
    lemmas, inflected_words, pairs_tags = readLangs(lang, setting) # S/E'
    print("Read %s lemma/word pairs" % len(pairs_tags)) # TUOP
    print("Finding maximum string length...") # TUOP
    max_length = 50 # S/E
    print("Maximum string length: %s" % max_length) # TUOP
    print("Counting lemmas/words...") # TUOP
    for pairtag in pairs_tags:
        lemmas.addWord(pairtag[0][0],pairtag[1]) # S/E
        inflected_words.addWord(pairtag[0][1]) # S/E
    print("Counted lemmas/words:") # TUOP
    print(lemmas.name, lemmas.n_chars) # TUOP
    print(inflected_words.name, inflected_words.n_chars) # TOUP
    return lemmas, inflected_words, pairs_tags, max_length # S/E


lemmas, inflected_words, pairs_tags, max_length = prepareData('navajo', 'train-low') # S/E
print(random.choice(pairs_tags)) # TUOP

Reading lines...
Read 100 lemma/word pairs
Finding maximum string length...
Maximum string length: 50
Counting lemmas/words...
Counted lemmas/words:
lemmas 57
inflected words 34
(['neilé', 'ndazhdooleeł'], ['V', 'REAL', 'PROSP', '4', 'PL'])


In [5]:
# Encoder RNN

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__() # S/E
        self.hidden_size = hidden_size # Sets size of hidden layer

        self.embedding = nn.Embedding(input_size, hidden_size) # Sets embedding layer
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True) # Sets bidirectional GRU

    def forward(self, input, hidden): # Computes forward propogation
        embedded = self.embedding(input).view(1, 1, -1) # Reshapes tensor
        output = embedded # S/E
        output, hidden = self.gru(output, hidden) # Runs output and hidden through GRU
        return output, hidden # S/E 

    def initHidden(self): # Used to initiate hidden layer dims
        return torch.zeros(2, 1, self.hidden_size, device=device) # Returns torch tensor of zeros in given dims


In [6]:
# Decoder RNN with Attention

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__() # S/E
        self.hidden_size = hidden_size # Sets hidden layer size
        self.output_size = output_size # Sets output layer size
        self.dropout_p = dropout_p # Sets dropout rate
        self.max_length = max_length # Sets maximum string length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size) # Sets embedding layer
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length) # Sets attention mechanism
        # Combines attention mecahanims parts?
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size) 
        self.dropout = nn.Dropout(self.dropout_p) # Implements dropout
        self.gru = nn.GRU(self.hidden_size, self.hidden_size) # Sets GRU
        self.out = nn.Linear(self.hidden_size, self.output_size) # Sets linear output function
    
    def forward(self, input, hidden, encoder_outputs): # Forward propogation
        embedded = self.embedding(input).view(1, 1, -1) # Reshapes embedding tensor for hidden layer shape
        embedded = self.dropout(embedded) # Implements dropout of embedded layer

        # Sets attention weights (read more)
        #    takes softmax of linear attention functions from concatenated embedding layer and hidden layer
        #    dim=1 is the dimension along which softmax is applied. why 0 index? (figure out later)
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        # Matrix multiplication of the attention weights and the encoder outputs
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        # ??? Concatenates embedding tensors and attention, but why? and why 0 index?
        output = torch.cat((embedded[0], attn_applied[0]), 1) 
        output = self.attn_combine(output).unsqueeze(0) # Figure all this attention stuff out later

        output = F.relu(output) # Runs output through ReLU function
        output, hidden = self.gru(output, hidden) # Runs output and hidden state through GRU

        # Runs output through linear function, then log softmax; why 0 index and why dim 1?
        output = F.log_softmax(self.out(output[0]), dim=1) 
        return output, hidden # S/E

    def initHidden(self): # Used to initiate hidden layer dims
        return torch.zeros(1, 1, self.hidden_size, device=device) # Returns torch tensor of zeros in given dims

In [7]:
# Creates tensors from indexes

def indexesFromWord(Lang, word): # Accesses character indexes of input word from Lang
    return [Lang.char2index[char] for char in word if char in Lang.char2index] # S/E

def tensorFromWord(Lang, word): # Creates tensor from input word
    indexes = indexesFromWord(Lang, word) # Gets list of indexes of characters in word
    indexes.append(EOW_token) # Adds end of word token
    # Returns tensor based on character indexes input
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1) # S/E

def indexesFromTags(Tags, tags): # Accesses tag indexes of input tags from Tags
    return [Tags.tag2index[tag] for tag in tags] # S/E

def tensorFromTags(Tags, tags):
    indexes = indexesFromTags(Tags, tags) # Gets list of indexes of characters in word
    # Returns tensor based on character indexes input
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1) # S/E

def tensorsFromPair(pair): # Creates tensor from lemma/word pair
    input_tensor = tensorFromWord(lemmas, pair[0][0]) # Creates input tensor
    target_tensor = tensorFromWord(inflected_words, pair[0][1]) # Creates target output tensor
    return (input_tensor, target_tensor) # S/E

In [8]:
# Training loop defintion

teacher_forcing_ratio = 0.5 # Sets probability of teacher forcing occuring

# Defines train function; following is each variable's function, in order:
#    input_tensor = input tensor
#    target_tensor = target tensor
#    encoder = instance of EncoderRNN
#    decoder = instance of AttnDecoderRNN
#    encoder_optimizer, decoder_optimizer = optimization algorithm (in this case SGD)
#    max_length = maximum string length
#    criterion = loss function (in this case negative log loss)
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer,\
          decoder_optimizer, max_length, criterion):
    encoder_hidden = encoder.initHidden() # Returns hidden layer dim for encoder

    encoder_optimizer.zero_grad() # Resets optimizer
    decoder_optimizer.zero_grad() # Resets optimizer

    input_length = input_tensor.size(0) # Input tensor length
    target_length = target_tensor.size(0) # Target tensor length

    # Sets encoder output dims
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size * 2, device=device) 

    loss = 0 # Sets/resets loss to 0

    for ei in range(input_length): # Tensor inputs
        # Calculates encoder output and hidden state based on input tensor and encoder hidden state
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0] # Stores encoder outputs (figure out later why these indexes)
    
    # Initiates decoder input with start of word token
    decoder_input = torch.tensor([[SOW_token]], device=device)
    
    decoder_hidden = encoder_hidden.view(1,1,-1) # Initiates decoder hidden state with final encoder hidden state

    # Randomly decide when to or not to use teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False 

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length): # Iterates through target tensors
            # Runs input, decoder hidden state, and encoder outputs through decoder
            #    and sets decoder output, hidden state, and attention to output of decoder
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di]) # Calculates loss
            # Teacher forcing: makes next input the target input instead of guessed output
            decoder_input = target_tensor[di]

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # Runs input, decoder hidden state, and encoder outputs through decoder
            #    and sets decoder output, hidden state, and attention to output of decoder
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1) # 
            # Keeps variable from backpropogating to the wrong place
            decoder_input = topi.squeeze().detach()  

            loss += criterion(decoder_output, target_tensor[di]) # Calculates loss
            if decoder_input.item() == EOW_token: # S/E
                break

    loss.backward() # S/E

    encoder_optimizer.step() # S/E
    decoder_optimizer.step() # S/E

    return loss.item() / target_length # S/E



In [9]:
# Creates timer functions; Not totally relevant

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [10]:
# Defines training iteration function

# Defines training iteration function; explanation of each input vairable:
#    encoder = instance of EncoderRNN
#    decoder = instance of AttnDecoderRNN
#    n_iters = number of iterations
#    print_every=# = print TUOP every # iterations
#    plot_every=# = plot TUOP every # iterations
#    learning_rate=# = S/E
def trainIters(encoder, decoder, n_iters, print_every=1000, learning_rate=0.01):
    start = time.time() # Starts timer
    plot_losses = [] # ??? Probably for plotting -- deal with later
    print_loss_total = 0  # Reset every print_every -- ??? I think: resets loss total to print

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate) # Sets encoder optimizer
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate) # Sets decoder optimizer
    # Sets up training data of tensors from 
    #    randomized selections of lemma/word pairs for the number of iterations
    training_pairs = [tensorsFromPair(random.choice(pairs_tags)) 
                      for i in range(n_iters)]
    criterion = nn.NLLLoss() # Sets loss function

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1] # Selects a training pair
        input_tensor = training_pair[0] # Takes input string's tensor
        target_tensor = training_pair[1] # Take target string's tensor

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, max_length, criterion) # S/E
        print_loss_total += loss # Adds current loss to total loss for printing

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every # Print avg of loss over iterations
            print_loss_total = 0 # Resets total loss to print
            # TUOP in terms of % complete and time taken, etc
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

In [11]:
### Evaluation code

# Defines evalutate:
#    encoder, decoder = instances of EncoderRNN, AttnDecoderRNN
#    lemma = word
#    max_length = maximum string length
def evaluate(encoder, decoder, lemma, max_length):
    with torch.no_grad(): # Keeps it from training
        input_tensor = tensorFromWord(lemmas, lemma) # Creates input tensor
        input_length = input_tensor.size()[0] # S/E
        encoder_hidden = encoder.initHidden() # Sets up hidden layer dims

        # Sets up encoder outputs dims and sets them to zeros
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size * 2, device=device)

        for ei in range(input_length): # Encoder input
            # Runs input tensor and encoder hidden state through encoder
            #    and sets encoder output and hidden state
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            # Adds encoder output to list of encoder outputs -- details later
            encoder_outputs[ei] += encoder_output[0, 0] 
            

        # Sets encoder input to tensor of start of word token
        decoder_input = torch.tensor([[SOW_token]], device=device)  

        decoder_hidden = encoder_hidden.view(1,1,-1) # Sets decoder hidden state to encoder hidden state
        
        decoded_chars = [] # Initiates list of decoded words
        # Sets decoder attentions
        #    To tensor of zeros of maximum string length as dimensions? Why those dims?
        #    Figure that out later
        decoder_attentions = torch.zeros(max_length, max_length) 

        for di in range(max_length): # Decoder input
            # Runs input tensor and decoder hidden state through decoder
            #    and sets decoder output and hidden state
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.data.topk(1) # Top tensor (top next char)
            if topi.item() == EOW_token: # Breaks if top word is end of word token
                decoded_chars.append('<EOW>')
                break
            else:
                # Appends top char to decoded word
                decoded_chars.append(inflected_words.index2char[topi.item()]) 
                
            decoder_input = topi.squeeze().detach() # Details later

        return decoded_chars # Returns decoded word

In [15]:
# Randomly evaluate words

def evaluateRandomly(encoder, decoder, pairs_tags, n=10): # n=# = number of samples to evaluate
    for i in range(n):
        pair = random.choice(pairs_tags)
        print('>', pair[0][0])
        print('=', pair[0][1])
        output_chars = evaluate(encoder, decoder, pair[0][0], max_length)
        output_word = ''.join(output_chars)
        print('<', output_word)
        print('')

In [53]:
hidden_size = 256 # Hidden layer size
# Initiates instance of EncoderRNN with input size of number of unique chars? and hidden size as above
encoder1 = EncoderRNN(lemmas.n_chars, hidden_size).to(device) 
# Initiates instance of AttnDecoderRNN with hidden size as above, output size of number of unique chars?
#    the maximum string length, and the dropout rate
attn_decoder1 = AttnDecoderRNN(hidden_size * 2, inflected_words.n_chars, max_length, dropout_p=0.1).to(device)

# Executes program
trainIters(encoder1, attn_decoder1, 75000, print_every=5000)


5m 3s (- 70m 47s) (5000 6%) 1.2022
10m 14s (- 66m 35s) (10000 13%) 0.1676
15m 27s (- 61m 50s) (15000 20%) 0.1211
20m 38s (- 56m 46s) (20000 26%) 0.1144
25m 48s (- 51m 37s) (25000 33%) 0.1103
30m 59s (- 46m 29s) (30000 40%) 0.1001
36m 13s (- 41m 23s) (35000 46%) 0.1001
41m 27s (- 36m 16s) (40000 53%) 0.0920
46m 35s (- 31m 3s) (45000 60%) 0.1010
51m 47s (- 25m 53s) (50000 66%) 0.0855
57m 1s (- 20m 44s) (55000 73%) 0.0933
62m 16s (- 15m 34s) (60000 80%) 0.0832
67m 29s (- 10m 23s) (65000 86%) 0.0939
72m 45s (- 5m 11s) (70000 93%) 0.0878
78m 24s (- 0m 0s) (75000 100%) 0.0854


In [54]:
evaluateRandomly(encoder1, attn_decoder1, pairs_tags) # Evaluates random samples

> hálátsíín
= danihílátsíín
< bílátííín<EOW>

> yiłdééh
= jííłdeeʼ
< jííłdeeʼ<EOW>

> yíłdóóh
= dadiildoh
< dadiildoh<EOW>

> aʼáád
= yiʼáád
< yiʼáád<EOW>

> yiniʼįįh
= jineezʼį́į́ʼ
< jineezʼį́į́ʼ<EOW>

> ííłdįįh
= ádasiildįįd
< ádasiildįįd<EOW>

> yitłʼó
= yiitłʼó
< yiitłʼó<EOW>

> naatłíísh
= ndeiitłíísh
< ndeiitłíísh<EOW>

> yiłdin
= yishdin
< yishdin<EOW>

> łíidilchí
= łídadołchí
< łídadołchí<EOW>



In [55]:
# Implements evaluation of system

cor = 0
inc = 0
for pair, tags in pairs_tags:
    output_chars = evaluate(
        encoder1, attn_decoder1, pair[0], max_length) # Outputs word from system given input word
    print('>', pair[0])
    print('=', pair[1])
    print('<', "".join(output_chars[:-1]))
    if "".join(output_chars[:-1]) == pair[1]:
        print("cor")
        cor += 1
    else:
        print("inc")
        inc += 1
    print("\n")
print("Accuracy:", cor/(cor+inc))

> yidlą́
= joodlą́ą́ʼ
< joodlą́ą́ʼ
cor


> hálátsíín
= bílátsíín
< bílátííín
inc


> yitin
= yitin
< yitin
cor


> neilé
= nidasiilyá
< ndazhdooleeł
inc


> yiʼeeł
= yíʼéél
< yíʼéél
cor


> yidiiłtłʼíísh
= dadiiltłʼíísh
< dadiiltłʼíísh
cor


> yishóóh
= wooshóóʼ
< wooshóóʼ
cor


> yijááh
= nijááh
< nijááh
cor


> yighaał
= dajighaał
< dajighaał
cor


> yitʼood
= yiitʼóód
< yiitʼóód
cor


> yiyiigish
= jizhgish
< jizhgish
cor


> yikʼęęh
= nikʼęęh
< nikʼęęh
cor


> naatłíísh
= ndeiitłíísh
< ndeiitłíísh
cor


> łíidilchí
= łídadołchí
< łídadołchí
cor


> yildééh
= dííldah
< dííldah
cor


> yiniʼįįh
= jineezʼį́į́ʼ
< jineezʼį́į́ʼ
cor


> tłʼaakał
= nihitłʼaakał
< nihitłʼaakał
cor


> neiłjooł
= ndadoołjoł
< ndadoołjoł
cor


> haniih
= hohniih
< hojiniih
inc


> bitseeʼ
= yitseeʼ
< yitseeʼ
cor


> beeldléí
= shibeeldléí
< shibeeldléí
cor


> yiibááh
= dayiibááh
< dayiibááh
cor


> yitłeeh
= deiitłeeh
< deiitłeeh
cor


> bitseeʼ
= atseeʼ
< yitseeʼ
inc


> akʼeʼelchí
= akʼeʼjishchį́
< akʼeʼji

In [61]:
dev_lemmas, dev_inflected_words, dev_pairs_tags, dev_max_length = prepareData('navajo', 'dev') # S/E
print(random.choice(pairs_tags)) # TUOP

Reading lines...
Read 1000 lemma/word pairs
Finding maximum string length...
Maximum string length: 50
Counting lemmas/words...
Counted lemmas/words:
lemmas 59
inflected words 36
(['aʼáád', 'yiʼáád'], ['N', 'PSS4', 'ARGAC3S'])


In [62]:
evaluateRandomly(encoder1,attn_decoder1,dev_pairs_tags) # Evaluates random samples

> yénálniih
= bénéiilniiʼ
< hojinih<EOW>

> neitłeeh
= nideiztłééʼ
< deiitłeeh<EOW>

> iitsʼǫǫd
= adadoohtsʼǫł
< yiitʼóód<EOW>

> yideeł
= daadeeł
< yíʼéél<EOW>

> yiniisííł
= jiniisiil
< adíílizh<EOW>

> haiłtʼeʼ
= hadeeshtʼeeł
< adasiiʼeeł<EOW>

> neishood
= ndeidooshoł
< deiilzood<EOW>

> haiłgéésh
= hadajiłgéésh
< hajishaas<EOW>

> yitłíísh
= dasiitłizh
< deiitłíísh<EOW>

> yileeł
= jooleeł
< yíʼéél<EOW>



In [63]:
# Implements evaluation of system on dev set

cor = 0
inc = 0
for pair, tags in dev_pairs_tags:
    output_chars = evaluate(
        encoder1, attn_decoder1, pair[0], max_length) # Outputs word from system given input word
    print('>', pair[0])
    print('=', pair[1])
    print('<', "".join(output_chars[:-1]))
    if "".join(output_chars[:-1]) == pair[1]:
        print("cor")
        cor += 1
    else:
        print("inc")
        inc += 1
    print("\n")
print("Accuracy:", cor/(cor+inc))

> dizheeh
= díízhééʼ
< deiitłeeh
inc


> yą́ą́ʼdíłgééd
= yą́ą́daʼdííłgeed
< yą́ą́dadiʼdoołgoł
inc


> yiłtʼá
= niłtʼá
< yitłʼáá
inc


> yiniisííł
= jiniisiil
< adíílitłʼíísh
inc


> yiłtłah
= yishtłah
< dajitłʼaah
inc


> neiyé
= nidajiyé
< neizyį́
inc


> iizǫ́ǫ́s
= adaahsǫ́ǫ́s
< niishyį́
inc


> yilʼaah
= joolʼaad
< dajikaʼ
inc


> haiłtʼeʼ
= hadeeshtʼeeł
< adasiiʼeeł
inc


> nayiiłniih
= nidahiilniih
< jiohinii
inc


> azází
= yizází
< bikáz
inc


> łeetsʼaaʼ
= nihiłeetsʼaʼ
< nihitsʼ
inc


> haniiʼ
= danihiniiʼ
< hanihiich
inc


> habid
= danihibid
< wadiichʼił
inc


> naalzheeh
= nidajishzheeʼ
< deiitłeeh
inc


> yiłnaad
= doołnał
< bigházhʼ
inc


> yiʼéésh
= jííʼeezh
< jiizhʼoéh
inc


> atsooʼ
= danihitsooʼ
< nihstsʼoł
inc


> neiłtseed
= naastseed
< yeeʼ
inc


> ííłdįįh
= ájíłdįįh
< ádasiildįįd
inc


> yizéés
= yiséés
< yiszyį́į́ʼ
inc


> kʼédǫǫh
= kʼídaazdǫǫd
< jioobish
inc


> hadoh
= nidoh
< ndeishoozh
inc


> shíniʼ
= bíniʼ
< danihichííʼ
inc


> yiyiiłchxosh
= daoołchxosh
< j

> yiyiighas
= siigas
< jizhgish
inc


> naalʼaʼ
= naashʼaʼ
< nihitsiʼ
inc


> naałchid
= nidaałchid
< ndadiichʼił
inc


> naaljooł
= ndooljoł
< ndadhooł
inc


> yiłdon
= yiildon
< yishdooł
inc


> yiłbal
= yiłbal
< bishdléí
inc


> yidzį́į́s
= dajidzį́į́s
< ndeiidzį́į́s
inc


> diichʼééh
= dadiichʼééh
< dajiiltłʼééh
inc


> neiłʼaʼ
= neiłʼaʼ
< dajitłʼ
inc


> naalzheeh
= ndadiilzhah
< deiitłeeh
inc


> yikʼąąs
= síníkʼą́ą́z
< jiłtsʼǫł
inc


> yijeeh
= deidoojah
< yiitseeh
inc


> yiyiizoh
= yizoh
< dajííyizh
inc


> hákʼaʼí
= níkʼaʼí
< nihikʼéí
inc


> nákwi
= nísiikwi
< nídeiikwi
inc


> naałchid
= nizhdoołchił
< ndadiichʼił
inc


> yiigááh
= jiigaii
< nijááh
inc


> yidą́
= jidą́
< joodlą́ą́ʼ
inc


> haikʼę́ę́h
= hajííkʼęęʼ
< haashkʼę́ę́h
inc


> álátsoh
= shílátsoh
< ndeishílz
inc


> neichʼid
= nishoochʼid
< ndadiichʼił
inc


> yiłchin
= niłchin
< yishdin
inc


> yibadooghááh
= bibadoónááh
< nijááh
inc


> yitsʼǫǫs
= jitsʼǫǫs
< yiitsʼǫs
inc


> naaʼnaʼ
= nidaaʼnaʼ
< nihitsʼ
inc


>

> amásání
= bimásání
< nihináísh
inc


> naʼachʼąąh
= niʼdoochʼąh
< jijichʼ
inc


> álátsíín
= yílátsíín
< bílitííítsíín
inc


> yishóóh
= deeshshoh
< wooshóóʼ
inc


> haikʼę́ę́h
= hayííkʼęęʼ
< haashkʼę́ę́h
inc


> álátsíín
= bílátsíín
< bílitííít
inc


> nádleeh
= nídaasdlį́į́ʼ
< deeildlee
inc


> naabaah
= nidasoobaaʼ
< hanihaash
inc


> haiłʼeeł
= hadajisʼéél
< adadooʼeł
inc


> yisdéíłʼeeł
= yisdádeiilʼeeł
< yííldaéł
inc


> yiyiiłhé
= sisxé
< jííyizh
inc


> yisééh
= jíísą́
< jilzh
inc


> yiyiizoh
= deiidzoh
< dajííyizh
inc


> yoolééł
= wohłééł
< yiidlééł
inc


> yisał
= yiilzał
< jidoołgił
inc


> naabé
= naabé
< ndadiioláe
inc


> yitʼood
= yítʼóód
< yiitʼóód
inc


> yooʼááł
= deínóhʼaah
< yiitłʼééh
inc


> neichʼid
= nanichʼid
< ndadiichʼił
inc


> alzhish
= daʼoolzhiizh
< hanishizh
inc


> yiyiiłtsóód
= jiiłtsood
< yiitʼóód
inc


> yitseeł
= dajitseeł
< yíítʼée doo
inc


> nilį́
= danilį́
< ndadiilą́ą́ʼ
inc


> yiyiitsʼǫs
= jidootsʼǫs
< yiitsʼǫs
inc


> atʼąąʼ
= shitʼąąʼ
< j

> yiiltééh
= yiiltééh
< diláhéét
inc


> naaʼnaʼ
= ndiiʼnah
< nihitsʼ
inc


> yigháázh
= wohháázh
< dajiáh
inc


> yiłchí
= shíníłchį́
< łídoołchí
inc


> achʼooní
= shichʼooní
< nihikʼéí
inc


> naałchid
= nidashoołchid
< ndadiichʼił
inc


> yiłnááh
= deiłnááh
< nijááh
inc


> yigháád
= yínígháád
< yiʼáád
inc


> yidiłhį́į́h
= diilyį́į́ʼ
< dadiilyį́į́ʼ
inc


> yikʼeed
= ńdeikʼiʼ
< yʼádeʼ
inc


> tłʼaajįʼééʼ
= atłʼaajįʼééʼ
< nihitłʼeeł
inc


> didzééh
= deedzaʼ
< dadeezdad
inc


> neiłté
= nisiiltį́
< ndadoołtłeeh
inc


> yilyé
= jidoolyééł
< yishyé
inc


> iimáás
= ajíímááz
< adasiilmááz
inc


> yiʼaał
= yiyííʼaal
< nihitłʼaakał
inc


> anázhiin
= yinázhiin
< ninih
inc


> ajaaʼ
= danihijaaʼ
< nihitłʼ
inc


> neiłgizh
= ndeidoołgish
< dadiibiz
inc


> nayiiłniih
= nahidiilnih
< jiohiniih
inc


> ashkii
= daniheʼashkii
< nihiikii
inc


> yiʼáád
= daooʼah
< yiʼáád
inc


> átááʼ
= nihítááʼ
< bítááʼ
inc


> yą́ą́ʼdíłgééd
= bą́ą́ʼdiilgeed
< yą́ą́dadiʼdoołgoł
inc


> ditłééʼ
= dinohtłééʼ
< 

> yikʼá
= wookʼą́
< nikʼáá
inc


> yootį́į́ł
= deíníitįįh
< woodlą́ą́ʼ
inc


> náʼáłkad
= náʼíłkad
< niʼáád
inc


> yíłmáás
= dadiilmas
< adasiilmááz
inc


> yénálniih
= yénáálniiʼ
< hojinih
inc


> neiloʼ
= ndazhdooloh
< nihitsiʼ
inc


> yiyiighas
= dajizghas
< jizhgish
inc


> iiłtsʼǫǫd
= adasiiltsʼǫ́ǫ́d
< yiitʼóód
inc


> yidiitsʼį́į́h
= dadidoohtsʼį́į́ł
< dadiilmááh
inc


> yiłchí
= yiłchí
< łídoołchí
inc


> naamaas
= ndoomas
< ndaah
inc


> haiłgééd
= haashgééd
< yą́ą́ʼadiʼoołgoł
inc


> neiłtseed
= ndoołtsił
< yeeʼ
inc


> iitʼááh
= iitʼááh
< nijááh
inc


> yookááł
= yishkááł
< nihʼééł
inc


> akʼeʼelchí
= akʼeʼdoołchííł
< akʼeʼjishchį́
inc


> sizį́
= daazį́
< ́ą́ą́ʼadiicoį́
inc


> heechééh
= hííchah
< dadeeʼádeeł
inc


> haiłhaał
= hadadoołhał
< hadajikaał
inc


> chʼínádzííd
= chʼééínídzid
< baʼádídloołgoł
inc


> ászólí
= ászólí
< bishildlí
inc


> atsiighaʼ
= bitsiighaʼ
< hinihtłʼ
inc


> yinizh
= wohnizh
< dajíínizh
inc


> haiłhaał
= haałhaał
< hadajikaał
inc


> yénálni